In [90]:
import cv2

img = cv2.imread('D:/Labeling_Server/dataUtil/model_dataset/model_learn_images/train/train_99_rotate0.jpg')

In [91]:
cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [92]:
th, tw = img.shape[:2]
print(th, tw)

1280 720


In [93]:
txt = '0 0.495703 0.564583 0.499219 0.479167'
txt = txt.split(' ')
txt

['0', '0.495703', '0.564583', '0.499219', '0.479167']

In [94]:
x, w = float(txt[1])*1280, float(txt[3])*1280 
y, h = float(txt[2])*720, float(txt[4])*720 
print(x,y,h,w)

634.49984 406.49976 345.00024 639.00032


In [95]:
nx, ny, nh, nw = int(x), int(y), int(h), int(w)
# cv2.rectangle(img, (nx, ny), (nx+nw, ny+nh), (0,0,255), 3)

# cv2.imshow('test', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [96]:
nx, ny = x - (w / 2), y - (h / 2)
nx, ny, h, w = int(nx), int(ny), int(h), int(w)
print(nx, ny, h, w)

314 233 345 639


In [ ]:
cv2.rectangle(img, (nx, ny), (nx+w, ny+h), (0,255,0), 3)

cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# crop 이슈 확인

In [1]:
import cv2

img = cv2.imread('D:/Labeling_Server/dataUtil/model_dataset/model_learn_images/train/train_0.jpg')
th, tw = img.shape[:2]

txt = '0 0.444141 0.512500 0.511719 0.486111'
txt = txt.split(' ')

In [2]:
rate = 0.15

start_x = int(tw * rate)
end_x = int(tw - start_x)
start_y = int(th * rate)
end_y = int(th - start_y)

print((start_x, end_x), (start_y, end_y))

(192, 1088) (108, 612)


In [3]:
import copy
# cr_img = copy.deepcopy(img[start_x:end_x, start_y:end_y])
cr_img = copy.deepcopy(img[start_y:end_y, start_x:end_x])

cv2.imshow('test', cr_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
nsize_h, nsize_w, _ = cr_img.shape
print(nsize_h, nsize_w)

crr_img = cv2.resize(cr_img, dsize=(tw, th), interpolation=cv2.INTER_AREA)
print(crr_img.shape)

504 896
(720, 1280, 3)


In [5]:
cv2.imshow('test', crr_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
x, y = float(txt[1]) * tw, float(txt[2]) * th
w, h = float(txt[3]) * tw, float(txt[4]) * th

x -= (w / 2)
y -= (h / 2)

bbox_x, bbox_y, bbox_h, bbox_w = int(x), int(y), int(h), int(w)
print(bbox_x, bbox_y, bbox_h, bbox_w)

241 194 349 655


In [7]:
nx, ny, nh, nw = bbox_x, bbox_y, bbox_h, bbox_w

start_nx = int(tw * rate)
start_ny = int(th * rate)

nx -= start_nx
ny -= start_ny

In [8]:
bbox_x = int((nx * (tw / nsize_w)))
bbox_y = int((ny * (th / nsize_h)))
bbox_h = int((nh * (th / nsize_h)))
bbox_w = int((nw * (tw / nsize_w)))

print(bbox_x, bbox_y, bbox_h, bbox_w)

70 122 498 935


In [10]:
cv2.rectangle(crr_img, (bbox_x, bbox_y), (bbox_x+bbox_w, bbox_y+bbox_h), (255,0,0), 3)

cv2.imshow('test', crr_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# rotate 이슈 확인

In [1]:
import cv2
import numpy as np

img = cv2.imread('D:/Labeling_Server/dataUtil/model_dataset/model_learn_images/train/train_105.jpg')

txt = '0 0.445703 0.511111 0.507031 0.480556'
txt = txt.split(' ')

angle = 90

cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
def yoloFormattocv(x1, y1, x2, y2, H, W):
    bbox_width = x2 * W
    bbox_height = y2 * H
    center_x = x1 * W
    center_y = y1 * H

    voc = []

    voc.append(center_x - (bbox_width / 2))
    voc.append(center_y - (bbox_height / 2))
    voc.append(center_x + (bbox_width / 2))
    voc.append(center_y + (bbox_height / 2))

    return [int(v) for v in voc]


def cvFormattoYolo(corner, H, W):
    bbox_W = corner[3] - corner[1]
    bbox_H = corner[4] - corner[2]

    center_bbox_x = corner[1] + (corner[3] / 2)
    center_bbox_y = corner[2] + (corner[4] / 2)

    return [corner[0], round(center_bbox_x / W, 6), round(center_bbox_y / H, 6), round(bbox_W / W, 6), round(bbox_H / H, 6)]

rotation_angle = angle * np.pi / 180
rot_matrix = np.array(
    [[np.cos(rotation_angle), -np.sin(rotation_angle)], [np.sin(rotation_angle), np.cos(rotation_angle)]])

height, width = img.shape[:2]
image_center = (width / 2, height / 2)

rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

abs_cos = abs(rotation_mat[0, 0])
abs_sin = abs(rotation_mat[0, 1])

bound_w = int(height * abs_sin + width * abs_cos)
bound_h = int(height * abs_cos + width * abs_sin)

rotation_mat[0, 2] += bound_w / 2 - image_center[0]
rotation_mat[1, 2] += bound_h / 2 - image_center[1]

rotated_mat = cv2.warpAffine(img, rotation_mat, (bound_w, bound_h))

new_height, new_width = rotated_mat.shape[:2]
H, W = img.shape[:2]

# cv2.imshow('test', rotated_mat)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [3]:
(minx, miny, maxx, maxy) = yoloFormattocv(
    float(txt[1]), float(txt[2]), float(txt[3]), float(txt[4]), H, W)

upper_left_corner_shift = (minx - W / 2, -H / 2 + miny)
upper_right_corner_shift = (maxx - W / 2, -H / 2 + miny)
lower_left_corner_shift = (minx - W / 2, -H / 2 + maxy)
lower_right_corner_shift = (maxx - W / 2, -H / 2 + maxy)

new_lower_right_corner = [-1, -1]
new_upper_left_corner = []

print(upper_left_corner_shift, upper_right_corner_shift, lower_left_corner_shift, lower_right_corner_shift)

(-394.0, -166.0) (254.0, -166.0) (-394.0, 181.0) (254.0, 181.0)


In [4]:
for i in (upper_left_corner_shift, upper_right_corner_shift, lower_left_corner_shift, lower_right_corner_shift):
    print('i:', i)
    new_coords = np.matmul(rot_matrix, np.array((i[0], -i[1])))
    print('new_coords:', new_coords)
    x_prime, y_prime = new_width / 2 + new_coords[0], new_height / 2 - new_coords[1]
    if new_lower_right_corner[0] < x_prime:
        new_lower_right_corner[0] = x_prime
    if new_lower_right_corner[1] < y_prime:
        new_lower_right_corner[1] = y_prime

    if len(new_upper_left_corner) > 0:
        if new_upper_left_corner[0] > x_prime:
            new_upper_left_corner[0] = x_prime
        if new_upper_left_corner[1] > y_prime:
            new_upper_left_corner[1] = y_prime
    else:
        new_upper_left_corner.append(x_prime)
        new_upper_left_corner.append(y_prime)

new_bbox = [txt[0], new_upper_left_corner[0], new_upper_left_corner[1], new_lower_right_corner[0], new_lower_right_corner[1]]
print(new_bbox)
# new_bbox = cvFormattoYolo(new_bbox, H, W)
# print(new_bbox)

i: (-394.0, -166.0)
new_coords: [-166. -394.]
i: (254.0, -166.0)
new_coords: [-166.  254.]
i: (-394.0, 181.0)
new_coords: [ 181. -394.]
i: (254.0, 181.0)
new_coords: [181. 254.]
['0', 193.99999999999997, 386.0, 541.0, 1034.0]


In [5]:
nx,ny,xx,xy = int(new_bbox[1]), int(new_bbox[2]), int(new_bbox[3]), int(new_bbox[4])

# w, h = xx - nx, xy - ny
# cx, cy = int(nx + w/2), int(ny + h/2)
# print(cx, cy, w, h)

cv2.rectangle(rotated_mat, (nx, ny), (xx, xy), (0, 255, 0), 3)

cv2.imshow('img', rotated_mat)
cv2.waitKey()
cv2.destroyAllWindows()

In [27]:
k = nx + 60
t = ny - 50

cv2.rectangle(rotated_mat, (k, t), (k+w, t+h), (0, 255, 0), 3)

cv2.imshow('img', rotated_mat)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
from math import cos, sin

angle = 90

h, w = img.shape[:2]
cx, cy = (w // 2, h // 2)

In [8]:
width = w * cos(angle) + h * sin(angle)
height = h * cos(angle) + w * sin(angle)

print(width, height)

center = (width / 2, height / 2)
print(center)

70.14336914706394 821.7027257957116
(35.07168457353197, 410.8513628978558)


# 정상 동작 확인

In [6]:
import cv2
import numpy as np

img = cv2.imread('D:/Projects/Labeling_Server/dataUtil/model_dataset/model_learn_images/train/train_102_crop0.jpg')
print(img.shape)

txt = '0 0.482422 0.606944 0.903906 0.786111'
txt = txt.split(' ')

cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

(720, 1280, 3)


In [7]:
th, tw = img.shape[:2]

x, y, w, h = float(txt[1]) * tw, float(txt[2]) * th, float(txt[3]) * tw, float(txt[4]) * th
print(x, y, w, h)

617.50016 436.99968 1156.99968 565.99992


In [8]:
x1, y1 = x - (w / 2), y - (h / 2)

x1, y1, w, h = int(x1), int(y1), int(w), int(h)

cv2.rectangle(img, (x1, y1), (x1+w, y1+h), (0,255,0), 3)
cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# test

In [1]:
# example of getting a child process by name
from time import sleep
from multiprocessing import active_children
from multiprocessing import Process
 
# return a child process with a given name or none
def get_process_by_name(name):
    # get all child processes
    processes = active_children()
    # return the process with the name
    for process in processes:
        # check for match
        if process.name == name:
            return process
    # no match
    return None
 
# function executed in a child process
def task():
    # block for a while
    sleep(3)
    print('sleep 3sec in task')
 
# protect the entry point
if __name__ == '__main__':
    # configure child processes
    children = [Process(target=task) for _ in range(10)]
    # start child processes
    for child in children:
        child.start()
    # wait a moment
    sleep(1)
    print('sleep 1sec')
    # get the child by name
    child = get_process_by_name('Process-1')
    print(f'Found: {child}')

sleep 1sec
Found: None
